# Libraries

In [3]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#from bs4 import BeautifulSoup 

import json # library to handle JSON files

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py36h5fab9bb_0 --> 2020.12.5-py36h5fab9bb_1



geopy-2.1.0          | 64 KB     

# Data: London, UK

In [6]:
#!pip install fsspec

     |████████████████████████████████| 102kB 15.6MB/s ta 0:00:01


In [10]:
london_df = pd.read_csv('https://files.datapress.com/london/dataset/london-borough-profiles/2015-09-24T15:49:52/london-borough-profiles.csv', 
                        sep=',', 
                        error_bad_lines=False,
                        encoding= 'unicode_escape')
london_df.head()

,Code,Area_name,Inner/_Outer_London,GLA_Population_Estimate_2017,GLA_Household_Estimate_2017,Inland_Area_(Hectares),Population_density_(per_hectare)_2017,"Average_Age,_2017","Proportion_of_population_aged_0-15,_2015","Proportion_of_population_of_working-age,_2015","Proportion_of_population_aged_65_and_over,_2015",Net_internal_migration_(2015),Net_international_migration_(2015),Net_natural_change_(2015),%_of_resident_population_born_abroad_(2015),Largest_migrant_population_by_country_of_birth_(2011),%_of_largest_migrant_population_(2011),Second_largest_migrant_population_by_country_of_birth_(2011),%_of_second_largest_migrant_population_(2011),Third_largest_migrant_population_by_country_of_birth_(2011),%_of_third_largest_migrant_population_(2011),%_of_population_from_BAME_groups_(2016),%_people_aged_3+_whose_main_language_is_not_English_(2011_Census),"Overseas_nationals_entering_the_UK_(NINo),_(2015/16)","New_migrant_(NINo)_rates,_(2015/16)",Largest_migrant_population_arrived_during_2015/16,Second_largest_migrant_population_arrived_during_2015/16,Third_largest_migrant_population_arrived_during_2015/16,Employment_rate_(%)_(2015),Male_employment_rate_(2015),Female_employment_rate_(2015),Unemployment_rate_(2015),Youth_Unemployment_(claimant)_rate_18-24_(Dec-15),Proportion_of_16-18_year_olds_who_are_NEET_(%)_(2014),Proportion_of_the_working-age_population_who_claim_out-of-work_benefits_(%)_(May-2016),%_working-age_with_a_disability_(2015),Proportion_of_working_age_people_with_no_qualifications_(%)_2015,Proportion_of_working_age_with_degree_or_equivalent_and_above_(%)_2015,"Gross_Annual_Pay,_(2016)",Gross_Annual_Pay_-_Male_(2016),Gross_Annual_Pay_-_Female_(2016),Modelled_Household_median_income_estimates_2012/13,%_adults_that_volunteered_in_past_12_months_(2010/11_to_2012/13),Number_of_jobs_by_workplace_(2014),%_of_employment_that_is_in_public_sector_(2014),"Jobs_Density,_2015","Number_of_active_businesses,_2015",Two-year_business_survival_rates_(started_in_2013),Crime_rates_per_thousand_population_2014/15,Fires_per_thousand_population_(2014),Ambulance_incidents_per_hundred_population_(2014),"Median_House_Price,_2015","Average_Band_D_Council_Tax_charge_(£),_2015/16",New_Homes_(net)_2015/16_(provisional),"Homes_Owned_outright,_(2014)_%","Being_bought_with_mortgage_or_loan,_(2014)_%","Rented_from_Local_Authority_or_Housing_Association,_(2014)_%","Rented_from_Private_landlord,_(2014)_%","%_of_area_that_is_Greenspace,_2005",Total_carbon_emissions_(2014),"Household_Waste_Recycling_Rate,_2014/15","Number_of_cars,_(2011_Census)","Number_of_cars_per_household,_(2011_Census)","%_of_adults_who_cycle_at_least_once_per_month,_2014/15","Average_Public_Transport_Accessibility_score,_2014","Achievement_of_5_or_more_A*-_C_grades_at_GCSE_or_equivalent_including_English_and_Maths,_2013/14",Rates_of_Children_Looked_After_(2016),%_of_pupils_whose_first_language_is_not_English_(2015),%_children_living_in_out-of-work_households_(2015),"Male_life_expectancy,_(2012-14)","Female_life_expectancy,_(2012-14)",Teenage_conception_rate_(2014),Life_satisfaction_score_2011-14_(out_of_10),Worthwhileness_score_2011-14_(out_of_10),Happiness_score_2011-14_(out_of_10),Anxiety_score_2011-14_(out_of_10),Childhood_Obesity_Prevalance_(%)_2015/16,People_aged_17+_with_diabetes_(%),Mortality_rate_from_causes_considered_preventable_2012/14,Political_control_in_council,Proportion_of_seats_won_by_Conservatives_in_2014_election,Proportion_of_seats_won_by_Labour_in_2014_election,Proportion_of_seats_won_by_Lib_Dems_in_2014_election,Turnout_at_2014_local_elections
0,E09000001,City of London,Inner London,8800,5326,290,30.3,43.2,11.4,73.1,15.5,-7,665,30,.,United States,2.8,France,2,Australia,1.9,27.5,17.1,975,152.2,India,France,United States,64.6,.,.,.,1.6,.,3.4,.,.,.,.,.,.,"£63,620",.,500400,3.4,84.3,26130,64.3,.,12.3,.,799999,931.2,80,.,.,.,.,4.8,1036,34.4,1692,0.4,16.9,7.9,78.6,101,.,7.9,.,.,.,6.6,7.1,6.0,5.6,NaN,2.6,129,.,.,.,.,.
1,E09000002,Barking and Dagenham,Outer London,209000,

In [28]:
CLIENT_ID = 'IHNVIXALFROMBOL3HQVFMZTLTCV4QGH0KTUJAY1G11GDIWBD' # your Foursquare ID
CLIENT_SECRET = 'DBQPKD5X4ZHDMUIGTPAGLY1A4ARV5YRLCM5DQY4R233V3TFD' # your Foursquare Secret
VERSION = '20190425' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
#ACCESS_TOKEN = 'OW2HYA0TFUZ3ERCUXOYEANOLILBRWWI1WGUP0TP2SBTUAVQ4'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IHNVIXALFROMBOL3HQVFMZTLTCV4QGH0KTUJAY1G11GDIWBD
CLIENT_SECRET:DBQPKD5X4ZHDMUIGTPAGLY1A4ARV5YRLCM5DQY4R233V3TFD


In [109]:
cities = ['London, SW3', 'London, SW5', 'London, SW6', 'London, SW7', 'London, SW10', 'London, W2', 'London, W6', 'London, W8', 'London, W9', 'London, W10', 'London, W11', 'London, W12', 'London, W14']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1d8941735") # GAY BAR CATEGORY ID
    results[city] = requests.get(url).json()
    
# https://developer.foursquare.com/docs/build-with-foursquare/categories/

In [113]:
results[city]['response']['geocode']

{'what': '',
 'where': 'london w14',
 'center': {'lat': 51.4938, 'lng': -0.2288},
 'displayString': 'W14, United Kingdom',
 'cc': 'GB',
 'longId': '149463212633404586'}

In [112]:
results[city]['response']['geocode']['center']['lat']

51.4938

In [116]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['center']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['center']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of LGBT places in {city} = ", results[city]['response']['totalResults'])

Total number of LGBT places in London, SW3 =  32
Total number of LGBT places in London, SW5 =  4
Total number of LGBT places in London, SW6 =  8
Total number of LGBT places in London, SW7 =  17
Total number of LGBT places in London, SW10 =  32
Total number of LGBT places in London, W2 =  4
Total number of LGBT places in London, W6 =  4
Total number of LGBT places in London, W8 =  4
Total number of LGBT places in London, W9 =  20
Total number of LGBT places in London, W10 =  4
Total number of LGBT places in London, W11 =  6
Total number of LGBT places in London, W12 =  4
Total number of LGBT places in London, W14 =  2


In [117]:
maps[cities[0]]

In [136]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['center']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['center']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="red", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#f67b6a',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])
    
    ave_dist = np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values))

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))
    print('------------------------------------')

London, SW3
Mean Distance from Mean coordinates
0.014845450943694459
------------------------------------
London, SW5
Mean Distance from Mean coordinates
0.011447021104808199
------------------------------------
London, SW6
Mean Distance from Mean coordinates
0.02670393897335906
------------------------------------
London, SW7
Mean Distance from Mean coordinates
0.004437251264176457
------------------------------------
London, SW10
Mean Distance from Mean coordinates
0.014845450943694459
------------------------------------
London, W2
Mean Distance from Mean coordinates
0.032228575379642625
------------------------------------
London, W6
Mean Distance from Mean coordinates
0.03162914450593372
------------------------------------
London, W8
Mean Distance from Mean coordinates
0.032228575379642625
------------------------------------
London, W9
Mean Distance from Mean coordinates
0.011623406634731503
------------------------------------
London, W10
Mean Distance from Mean coordinates
0.0

In [126]:
maps[cities[0]]

In [130]:
maps[cities[6]]

In [139]:
area = 'London, SW7'
venues_mean_coor = [df_venues[area]['Lat'].mean(), df_venues[area]['Lng'].mean()] 

print(area)
print("Mean Distance from Mean coordinates")
dists = np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[area][['Lat','Lng']].values)
dists.sort()
print(np.mean(dists[:-1]))# Ignore the biggest distance

London, SW7
Mean Distance from Mean coordinates
0.0032402762039001384
